In [4]:
import pandas as pd
from transformers import pipeline
from collections import defaultdict
import torch
from tqdm import tqdm
import numpy as np
import json
from datetime import datetime
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
from typing import List, Dict

# [Previous InstagramPostAnalyzer class remains the same]

def generate_analysis_report(results: List[Dict], output_dir: str = "analysis_output"):
    """Generate comprehensive analysis reports"""
    # Create output directory if it doesn't exist
    Path(output_dir).mkdir(parents=True, exist_ok=True)
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

    # Initialize DataFrames for different aspects
    topic_data = []
    sentiment_data = []
    engagement_data = []

    for post_analysis in results:
        sheet_name = post_analysis['sheet']

        # Topic analysis
        for topic, percentage in post_analysis['comments_analysis']['topics'].items():
            topic_data.append({
                'Sheet': sheet_name,
                'Topic': topic,
                'Percentage': percentage
            })

        # Sentiment analysis
        sentiment_data.append({
            'Sheet': sheet_name,
            'Positive_Sentiment': post_analysis['comments_analysis']['sentiment']['positive_percentage'],
            'Total_Comments': post_analysis['comments_analysis']['total_comments'],
            'Emoji_Usage': post_analysis['comments_analysis']['emoji_usage']['emoji_usage_percentage']
        })

    # Create DataFrames
    topic_df = pd.DataFrame(topic_data)
    sentiment_df = pd.DataFrame(sentiment_data)

    # Save to Excel with multiple sheets
    with pd.ExcelWriter(f"{output_dir}/analysis_report_{timestamp}.xlsx") as writer:
        topic_df.to_excel(writer, sheet_name='Topic Analysis', index=False)
        sentiment_df.to_excel(writer, sheet_name='Sentiment Analysis', index=False)

    # Generate visualizations
    plt.figure(figsize=(12, 6))
    sns.barplot(data=topic_df, x='Percentage', y='Topic')
    plt.title('Topic Distribution Across All Comments')
    plt.tight_layout()
    plt.savefig(f"{output_dir}/topic_distribution_{timestamp}.png")
    plt.close()

    # Generate JSON report
    report = {
        'analysis_timestamp': timestamp,
        'overall_statistics': {
            'total_posts_analyzed': len(results),
            'average_sentiment': sentiment_df['Positive_Sentiment'].mean(),
            'average_emoji_usage': sentiment_df['Emoji_Usage'].mean()
        },
        'detailed_results': results
    }

    with open(f"{output_dir}/detailed_report_{timestamp}.json", 'w') as f:
        json.dump(report, f, indent=2)

    return report


In [6]:
import re
from typing import Dict, List
from transformers import pipeline

In [7]:
class InstagramPostAnalyzer:
    def __init__(self):
        # Initialize sentiment analyzer
        self.sentiment_analyzer = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")

        # Initialize topic classifier
        self.topic_classifier = pipeline("text-classification", model="facebook/bart-large-mnli")

        # Define common topics for classification
        self.topics = [
            "product feedback",
            "customer service",
            "technical issue",
            "appreciation",
            "question",
            "suggestion",
            "complaint",
            "general discussion"
        ]

    def load_excel_data(self, file_path: str) -> List[Dict]:
        """
        Load and parse Excel file containing Instagram post data
        Returns a list of dictionaries containing data from each sheet
        """
        excel_file = pd.ExcelFile(file_path)
        posts_data = []

        for sheet_name in excel_file.sheet_names:
            df = pd.read_excel(excel_file, sheet_name=sheet_name)

            # Clean and prepare data
            df = df.fillna('')  # Replace NaN with empty string

            posts_data.append({
                'sheet': sheet_name,
                'comments': df['Comments'].tolist() if 'Comments' in df.columns else [],
                'likes': df['Likes'].tolist() if 'Likes' in df.columns else [],
                'dates': df['Date'].tolist() if 'Date' in df.columns else []
            })

        return posts_data

    def analyze_sentiment(self, text: str) -> Dict:
        """Analyze sentiment of given text"""
        try:
            result = self.sentiment_analyzer(text)[0]
            return {
                'label': result['label'],
                'score': result['score']
            }
        except Exception as e:
            print(f"Error analyzing sentiment for text: {e}")
            return {'label': 'NEUTRAL', 'score': 0.5}

    def classify_topic(self, text: str) -> str:
        """Classify the topic of given text"""
        try:
            # Create hypothesis template for zero-shot classification
            hypothesis_template = "This comment is about {}."

            result = self.topic_classifier(
                text,
                candidate_labels=self.topics,
                hypothesis_template=hypothesis_template
            )

            return result['labels'][0]  # Return most likely topic
        except Exception as e:
            print(f"Error classifying topic for text: {e}")
            return "general discussion"

    def count_emojis(self, text: str) -> int:
        """Count number of emojis in text using regex"""
        emoji_pattern = re.compile("["
            u"\U0001F600-\U0001F64F"  # emoticons
            u"\U0001F300-\U0001F5FF"  # symbols & pictographs
            u"\U0001F680-\U0001F6FF"  # transport & map symbols
            u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
            u"\U00002702-\U000027B0"
            u"\U000024C2-\U0001F251"
            "]+", flags=re.UNICODE)

        return len(emoji_pattern.findall(text))

    def analyze_post(self, post_data: Dict) -> Dict:
        """
        Analyze a single post's comments for sentiment, topics, and engagement
        Returns dictionary with analysis results
        """
        comments = post_data['comments']
        total_comments = len(comments)

        if total_comments == 0:
            return {
                'sheet': post_data['sheet'],
                'comments_analysis': {
                    'total_comments': 0,
                    'sentiment': {'positive_percentage': 0},
                    'topics': defaultdict(float),
                    'emoji_usage': {'emoji_usage_percentage': 0}
                }
            }

        # Initialize counters
        positive_sentiments = 0
        topic_counts = defaultdict(int)
        total_emojis = 0

        # Analyze each comment
        for comment in tqdm(comments, desc="Analyzing comments"):
            if not isinstance(comment, str):
                continue

            # Sentiment analysis
            sentiment = self.analyze_sentiment(comment)
            if sentiment['label'] == 'POSITIVE':
                positive_sentiments += 1

            # Topic classification
            topic = self.classify_topic(comment)
            topic_counts[topic] += 1

            # Emoji counting
            total_emojis += self.count_emojis(comment)

        # Calculate percentages
        topic_percentages = {
            topic: (count / total_comments * 100)
            for topic, count in topic_counts.items()
        }

        return {
            'sheet': post_data['sheet'],
            'comments_analysis': {
                'total_comments': total_comments,
                'sentiment': {
                    'positive_percentage': (positive_sentiments / total_comments * 100)
                },
                'topics': dict(topic_percentages),
                'emoji_usage': {
                    'emoji_usage_percentage': (total_emojis / total_comments * 100)
                }
            }
        }

In [8]:

def main(file_path: str):
    analyzer = InstagramPostAnalyzer()
    print("Loading data from Excel file...")
    posts_data = analyzer.load_excel_data(file_path)

    results = []
    for post_data in posts_data:
        analysis = analyzer.analyze_post(post_data)
        results.append(analysis)

        # Print immediate results
        print(f"\n=== Results for {post_data['sheet']} ===")
        print("\nTop Topics in Comments:")
        sorted_topics = sorted(
            analysis['comments_analysis']['topics'].items(),
            key=lambda x: x[1],
            reverse=True
        )[:5]  # Top 5 topics

        for topic, percentage in sorted_topics:
            print(f"{topic}: {percentage:.1f}%")

        print(f"\nEngagement Statistics:")
        print(f"Total Comments: {analysis['comments_analysis']['total_comments']}")
        print(f"Positive Sentiment: {analysis['comments_analysis']['sentiment']['positive_percentage']:.1f}%")
        print(f"Emoji Usage: {analysis['comments_analysis']['emoji_usage']['emoji_usage_percentage']:.1f}%")

    # Generate comprehensive report
    report = generate_analysis_report(results)
    return report

if __name__ == "__main__":
    file_path = "Netflix.xlsx"
    results = main(file_path)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

...

Error classifying topic for text: PreTrainedTokenizerFast._batch_encode_plus() got an unexpected keyword argument 'candidate_labels'
Error classifying topic for text: PreTrainedTokenizerFast._batch_encode_plus() got an unexpected keyword argument 'candidate_labels'
Error classifying topic for text: PreTrainedTokenizerFast._batch_encode_plus() got an unexpected keyword argument 'candidate_labels'
Error classifying topic for text: PreTrainedTokenizerFast._batch_encode_plus() got an unexpected keyword argument 'candidate_labels'


Analyzing comments:  97%|█████████▋| 110/113 [00:08<00:00, 19.74it/s]

Error classifying topic for text: PreTrainedTokenizerFast._batch_encode_plus() got an unexpected keyword argument 'candidate_labels'
Error classifying topic for text: PreTrainedTokenizerFast._batch_encode_plus() got an unexpected keyword argument 'candidate_labels'
Error classifying topic for text: PreTrainedTokenizerFast._batch_encode_plus() got an unexpected keyword argument 'candidate_labels'
Error classifying topic for text: PreTrainedTokenizerFast._batch_encode_plus() got an unexpected keyword argument 'candidate_labels'


Analyzing comments: 100%|██████████| 113/113 [00:08<00:00, 13.35it/s]

Error classifying topic for text: PreTrainedTokenizerFast._batch_encode_plus() got an unexpected keyword argument 'candidate_labels'
Error classifying topic for text: PreTrainedTokenizerFast._batch_encode_plus() got an unexpected keyword argument 'candidate_labels'

=== Results for Sheet1 ===

Top Topics in Comments:
general discussion: 100.0%

Engagement Statistics:
Total Comments: 113
Positive Sentiment: 54.9%
Emoji Usage: 65.5%
